# EngMeta

[EngMeta](https://doi.org/10.18419/darus-500) is a metadata Schema developed as part of the NFDI4Ing (Nationale Forschungsdaten Infrastruktur für Ingenieure).

The schema is written a .xsd-file (XML Schema file), however, the toolbox does not provide a converter into YAML file in order to use the schema as a convention. In this example, most mandatory and required fields of the EngMeta schema are manually translated into a convention-YAML file. The below table gives an overview of fields. **Note,** that not all fields are put into the YAML file as numerous attributes like the file size, file type etc. can be derived from the file itself and don't need to be provided by the user.

The fields are:

| Category   | Title   |      Standard Attribute Name      |  Data Type | Obligation | 
|----------|----------|:-------------:|------:|------:|
| **Descriptive Metadata** | Contact Person |  contact | \\$personOrOrganization | M |
| | Producer/Author |  creator | \\$personOrOrganization |M |
| | Contributor |  contributor | \\$personOrOrganization | O |
| | Title |  title | \\$str |M |
| | Description |  description | \\$str | O |
| | Keywords |  keywords | \\$str |R |
| | Subject |  subject | \\$str |R |
| | Dates (Creation, Publication, ...) |  dates | \\$str |R |
| **Process Metadata** | Provenance information | provenance | \\$processingStep |O |
| **Technical Metadata**| PID | identifier | \\$pid | M |
|  | Legal Information | rightsStatement | \\$rightsStatement | R |

In [1]:
import h5rdmtoolbox as h5tbx
cv = h5tbx.conventions.from_yaml('EngMeta.yaml')
cv

Convention("EngMeta")
contact: https://orcid.org/0000-0001-8729-0482
  File.__init__():
    * contact:
		Name and contact information of a person or organization that can handle requiries about the data over a long time period.
    * creator:
		Name and affiliation of the author(s) of the data. These are the persons who created the data.
    * pid:
		Persistent identifier as URI, can ge further specified through a type and a scheme.
    * title:
		A name or title by which a resource is known.
    * contributor (default=None):
		Name and affiliation of all persons who contributed to the data.
    * description (default=None):
		All additional information that does not fit in any of the other categories. May be used for technical information. It is a best practice to supply a description.
    * keywords (default=None):
		List of describing keywords.
    * dates (default=None):
		Dates according to the data.
    * subject (default=None):
		List of subjects the data belongs to.
    * prove

In [2]:
h5tbx.use(cv)

using("EngMeta")

In [3]:
contact = cv.registered_standard_attributes['contact']
contact.validator.model_fields

{'name': FieldInfo(annotation=str, required=True),
 'givenName': FieldInfo(annotation=str, required=False),
 'familyName': FieldInfo(annotation=str, required=False),
 'address': FieldInfo(annotation=str, required=False),
 'affiliation': FieldInfo(annotation=str, required=False),
 'email': FieldInfo(annotation=str, required=False),
 'id': FieldInfo(annotation=str, required=False),
 'role': FieldInfo(annotation=contributorType, required=False)}

In [4]:
contact.validate(dict(name='Matthias Probst',
                             id='https://orcid.org/0000-0001-8729-0482',
                             role='Researcher'
                             ), None)

personOrOrganization(name='Matthias Probst', givenName=None, familyName=None, address=None, affiliation=None, email=None, id='https://orcid.org/0000-0001-8729-0482', role=contributorType(value=<contributorType.Researcher: 'Researcher'>))

In [5]:
contact.validate(dict(name='Matthias Probst',
                             ), None)

personOrOrganization(name='Matthias Probst', givenName=None, familyName=None, address=None, affiliation=None, email=None, id=None, role=None)

In [10]:
with h5tbx.File(contact=dict(name='Matthias Probst'),
                creator=dict(name='Matthias Probst',
                             id='https://orcid.org/0000-0001-8729-0482',
                             role='Researcher'
                             ),
                pid=dict(id='123', type='other'),
                title='Test file to demonstrate usage of EngMeta schema') as h5:
    fname = h5.hdf_filename
    h5.dump()

## Mapping functions

Some metadata can be extracted automatically from the file, like the `file size`, `file type` and `check sum` for example. Such functions are needed, if metadata like this is required:

In [11]:
import hashlib
def extract_metadata(filename):
    with h5tbx.File(filename) as h5:
        fsize = h5.filesize
    
    return dict(file_size=fsize, file_type='hdf5', checksum=hashlib.md5(open(fname, 'rb').read()).hexdigest())

In [12]:
extract_metadata(fname)

{'file_size': array(10552) <Unit('byte')>,
 'file_type': 'hdf5',
 'checksum': '72a399d75914328a923e8d22c3276270'}